INSERT INTO `Team` (`id`,`name`,`shortname`,`stadium`,`founded`) VALUES (999,'Equipe inconnue', 'Inconnu','Stade inconnu', 1900);
 Valeur par défaut pour la table Team en cas de suppression ou d'absence d'équipe

In [8]:
import json
import pandas as pd
import numpy as np
import mysql.connector as connector
import datetime as dt
from mysql.connector import Error
from dotenv import dotenv_values
import os.path
import ast
import pymongo
import locale
import re

In [9]:
locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')

'fr_FR.UTF-8'

In [43]:
config = dotenv_values('../.env')

In [3]:
df_teams_ligue2 = pd.read_json('../data/teams/ligue2_2022-2024.json', orient='index')
df_teams_ligue2.reset_index(inplace=True)

In [4]:
df_teams_ligue2.rename(columns={ 'index' : 'name' }, inplace=True)
df_teams_ligue2

,name,fondation,stade,shortname
0,Le Havre Athletic Club (football),1884,Stade Océane,Le Havre AC
1,Football Club de Metz,1919,Stade Saint-Symphorien,FC Metz
2,Football Club des Girondins de Bordeaux,1920,Bordeaux Atlantique,Girondins de Bordeaux
3,Stade Malherbe Caen,1913,Stade Michel-d'Ornano,SM Caen
4,En avant de Guingamp,1912,Stade de Roudourou,EA Guingamp
5,Paris Football Club,1969,Stade Charléty,Paris FC
6,Association sportive de Saint-Étienne,1919,Stade Geoffroy-Guichard,AS Saint-Étienne
7,Football Club Sochaux-Montbéliard,1928,Stade Auguste-Bonal,FC Sochaux-Montbéliard
8,Grenoble Foot 38,1911,Stade des Alpes,Grenoble Foot 38
9,Union sportive Quevilly Rouen Métropole,1902,Stade Robert-Diochon,US Quevilly-Rouen


In [5]:
df_teams_ligue2['name'] = df_teams_ligue2['name'].str.replace(r' \((.*)\)', "", regex=True)
df_teams_ligue2['stade'] = df_teams_ligue2['stade'].str.replace(r'\[(.*)\]', "", regex=True)

df_teams_ligue2

,name,fondation,stade,shortname
0,Le Havre Athletic Club,1884,Stade Océane,Le Havre AC
1,Football Club de Metz,1919,Stade Saint-Symphorien,FC Metz
2,Football Club des Girondins de Bordeaux,1920,Bordeaux Atlantique,Girondins de Bordeaux
3,Stade Malherbe Caen,1913,Stade Michel-d'Ornano,SM Caen
4,En avant de Guingamp,1912,Stade de Roudourou,EA Guingamp
5,Paris Football Club,1969,Stade Charléty,Paris FC
6,Association sportive de Saint-Étienne,1919,Stade Geoffroy-Guichard,AS Saint-Étienne
7,Football Club Sochaux-Montbéliard,1928,Stade Auguste-Bonal,FC Sochaux-Montbéliard
8,Grenoble Foot 38,1911,Stade des Alpes,Grenoble Foot 38
9,Union sportive Quevilly Rouen Métropole,1902,Stade Robert-Diochon,US Quevilly-Rouen


In [6]:
df_teams_ligue2.dtypes

name         object
fondation     int64
stade        object
shortname    object
dtype: object

In [8]:
team_names = []
team_shortnames = []
team_founded = []
team_stadium = []
team_coaches = []
team_players = []

for i in range(11, 77):
    if os.path.isfile(f"../data/teams/team-5{i}.json"):
        with open(f'../data/teams/team-5{i}.json', 'r') as file:
            data = json.load(file)
            if 'errorCode' not in data:
                if data['shortName'] not in ['Le Havre', 'Saint-Étienne', 'Auxerre', 'Angers SCO']:
                    team_names.append(data['name'])
                    team_shortnames.append(data['shortName'])
                    team_founded.append(data['founded'])
                    team_stadium.append(data['venue'])
                    team_coaches.append(data['coach'])
                    team_players.append(data['squad'])

In [9]:
try:
    cnx = connector.connect(host='localhost',
                            database='football_predictor',
                            user=config['MYSQL_USER'],
                            password=config['MYSQL_PASSWORD'],
                            )
    if cnx.is_connected():
        print('Connecté à la base MySQL')
        cursor = cnx.cursor()
        # Chargement des données dans la base données SQL
        for i in range(len(team_names)):
            add_team_ligue1 = ( "INSERT INTO `Team` (name, shortname, stadium, founded) "
                                f"VALUES (\"{team_names[i]}\", \"{team_shortnames[i]}\", \"{team_stadium[i]}\", {team_founded[i]})" )
            cursor.execute(add_team_ligue1)
        for index, row in df_teams_ligue2.iterrows():
            add_team_ligue2 = ( "INSERT INTO `Team` (name, shortname, stadium, founded) "
                                f"VALUES (\"{row['name']}\", \"{row['shortname']}\", \"{row['stade']}\", {row['fondation']})" )
            cursor.execute(add_team_ligue2)
        cnx.commit()
except Error as e:
    cnx.close()
    print("Erreur lors de la connexion à la base MySQL :", e)
finally:
    if cnx.is_connected():
        cnx.close()
        print('Connexion à la base MySQL fermée')

Connecté à la base MySQL
Connexion à la base MySQL fermée


In [14]:
df_coaches_ligue2 = pd.read_json('../data/teams/coaches_ligue2_2022-2024.json', orient='index')
df_coaches_ligue2.reset_index(inplace=True)
df_coaches_ligue2

,index,surname,name,birthdate,nationality
0,Le Havre Athletic Club (football),Didier,Digard,1986-07-12,Française
1,Football Club de Metz,Stéphane,Le Mignan,1974-06-04,Français
2,Football Club des Girondins de Bordeaux,Bruno,Irles,1975-08-16,Française
3,Stade Malherbe Caen,Bruno,Baltazar,1977-07-06,Portugaise
4,En avant de Guingamp,Sylvain,Ripoll,1971-08-15,Français
5,Paris Football Club,Stéphane,Gilli,1974-04-30,Français
6,Association sportive de Saint-Étienne,Eirik,Horneland,1975-03-14,Norvégienne
7,Football Club Sochaux-Montbéliard,Frédéric,Bompard,1962-12-30,Français
8,Grenoble Foot 38,Franck,Rizzetto,1971-03-29,Français
9,Union sportive Quevilly Rouen Métropole,David,Carré,1971-03-29,Français


In [15]:
def nationality_formatter(nationality):
    dictionnary = { 'Français' : 'France',
                'Française' : 'France',
                'Sénégalaise' : 'Senegal',
                'Portugaise' : 'Portugal',
                'Portugais' : 'Portugal',
                'Marocaine' : 'Morocco',
                'Norvégienne' : 'Norway',
                'Algérienne' : 'Algeria' }
    nat_list = []
    for nat in nationality.split(' '):
        nat_list.append(dictionnary[nat])
    if len(nat_list) > 1 :
        nationality_formatted = ' '.join(nation for nation in nat_list)
    else:
        nationality_formatted = nat_list[0]
    return nationality_formatted

In [16]:
df_coaches_ligue2['nationality'] = df_coaches_ligue2['nationality'].apply(lambda x: str(x).replace(u'\xa0', u''))
df_coaches_ligue2['nationality'] = df_coaches_ligue2['nationality'].apply(lambda x: str(x).replace(u'/', u' '))
df_coaches_ligue2['nationality'] = df_coaches_ligue2['nationality'].apply(nationality_formatter)

In [17]:
df_coaches_ligue2['index'] = df_coaches_ligue2['index'].str.replace(r' \((.*)\)', "", regex=True)
df_coaches_ligue2

,index,surname,name,birthdate,nationality
0,Le Havre Athletic Club,Didier,Digard,1986-07-12,France
1,Football Club de Metz,Stéphane,Le Mignan,1974-06-04,France
2,Football Club des Girondins de Bordeaux,Bruno,Irles,1975-08-16,France
3,Stade Malherbe Caen,Bruno,Baltazar,1977-07-06,Portugal
4,En avant de Guingamp,Sylvain,Ripoll,1971-08-15,France
5,Paris Football Club,Stéphane,Gilli,1974-04-30,France
6,Association sportive de Saint-Étienne,Eirik,Horneland,1975-03-14,Norway
7,Football Club Sochaux-Montbéliard,Frédéric,Bompard,1962-12-30,France
8,Grenoble Foot 38,Franck,Rizzetto,1971-03-29,France
9,Union sportive Quevilly Rouen Métropole,David,Carré,1971-03-29,France


In [18]:
team_coaches

[{'id': 210420,
  'firstName': '',
  'lastName': 'Carles Martínez',
  'name': 'Carles Martínez',
  'dateOfBirth': '1984-01-01',
  'nationality': 'Spain',
  'contract': {'start': '2023-07', 'until': '2025-06'}},
 {'id': 190990,
  'firstName': 'Éric',
  'lastName': 'Roy',
  'name': 'Eric Roy',
  'dateOfBirth': '1967-09-26',
  'nationality': 'France',
  'contract': {'start': '2023-01', 'until': '2025-06'}},
 {'id': 11109,
  'firstName': 'Roberto',
  'lastName': 'De Zerbi',
  'name': 'Roberto De Zerbi',
  'dateOfBirth': '1979-06-06',
  'nationality': 'Italy',
  'contract': {'start': '2024-07', 'until': '2027-06'}},
 {'id': 45105,
  'firstName': 'Jean-Louis',
  'lastName': 'Gasset',
  'name': 'Jean-Louis Gasset',
  'dateOfBirth': '1953-12-09',
  'nationality': 'France',
  'contract': {'start': '2024-10', 'until': '2025-06'}},
 {'id': 45101,
  'firstName': 'Bruno',
  'lastName': 'Génésio',
  'name': 'Bruno Génésio',
  'dateOfBirth': '1966-09-01',
  'nationality': 'France',
  'contract': {'st

In [19]:
def dataframe_tuple_deserialize(string : str, param : int):
    tuple_string = ast.literal_eval(string)
    return tuple_string[param]

In [20]:
def correct_nationality(string : str):
    if string == "C%C3%B4te d%27Ivoire" :
        return 'Ivory Coast'
    elif string == "the Democratic Republic of the Congo" :
        return 'DR Congo'
    elif string == "the Republic of the Congo":
        return 'Congo'
    else:
        return string

In [21]:
df_list = []
for team in df_coaches_ligue2['index']:
    df_players_test = pd.read_csv(f'../data/teams/team-{team.replace(' ', '_')}.csv')
    df_players_test = df_players_test[df_players_test.columns[2:6]]
    df_players_test.dropna(inplace=True)
    df_players_test = df_players_test[df_players_test['Nom'] != "('', None)"]
    # Colonne Poste (P.)
    df_players_test[df_players_test.columns[0]] = df_players_test[df_players_test.columns[0]].apply(dataframe_tuple_deserialize, args=(0,))
    dictionnaire_poste = { 'G' : 'Gardien', 'D' : 'Defenseur', 'M' : 'Milieu', 'A' : 'Attaquant'}
    df_players_test[df_players_test.columns[0]] = df_players_test[df_players_test.columns[0]].replace(dictionnaire_poste)
    # Colonne Nationality
    df_players_test[df_players_test.columns[1]] = df_players_test[df_players_test.columns[1]].apply(dataframe_tuple_deserialize, args=(1,))
    df_players_test[df_players_test.columns[1]] = df_players_test[df_players_test.columns[1]].apply(lambda x : x.split('Flag_of_', 1)[1])
    df_players_test[df_players_test.columns[1]] = df_players_test[df_players_test.columns[1]].str.replace(r'_\((.*)\)', "", regex=True)
    df_players_test[df_players_test.columns[1]] = df_players_test[df_players_test.columns[1]].str.replace(".svg", "")
    df_players_test[df_players_test.columns[1]] = df_players_test[df_players_test.columns[1]].str.replace("_", " ")
    df_players_test[df_players_test.columns[1]] = df_players_test[df_players_test.columns[1]].apply(correct_nationality)
    # Colonne Nom
    df_players_test[df_players_test.columns[2]] = df_players_test[df_players_test.columns[2]].apply(dataframe_tuple_deserialize, args=(0,))
    df_players_test[['Prénom', 'Nom']] = df_players_test['Nom'].str.split(' ', n=1, expand=True)
    # Colonne Date de naissance
    df_players_test[df_players_test.columns[3]] = df_players_test[df_players_test.columns[3]].apply(dataframe_tuple_deserialize, args=(0,))
    df_players_test[df_players_test.columns[3]] = df_players_test[df_players_test.columns[3]].str.replace(r'\xa0\((.*)\)', "", regex=True)
    df_players_test[df_players_test.columns[3]] = pd.to_datetime(df_players_test[df_players_test.columns[3]], format="%d/%m/%Y")
    # Colonne equipe
    df_players_test['Equipe'] = team
    df_list.append(df_players_test)
df_list

[           P.    Nat.[153]            Nom Date de naissance     Prénom  \
 0     Gardien       France       Gorgelin        1990-08-05    Mathieu   
 1     Gardien       France         Desmas        1994-04-07     Arthur   
 3   Defenseur       France         Lloris        1995-07-18    Gautier   
 4   Defenseur       France  Youté Kinkoué        2002-01-14    Étienne   
 5   Defenseur      Hungary           Nego        1991-01-15       Loïc   
 6   Defenseur       France         Bodmer        2004-05-06     Mathéo   
 7   Defenseur      Morocco      El Hajjam        1991-02-19     Oualid   
 8   Defenseur       France        Zouaoui        1998-04-28      Yanis   
 9   Defenseur       France        Pembélé        2002-09-09   Timothée   
 10  Defenseur      Senegal       Sangante        2002-04-12     Arouna   
 11  Defenseur      Senegal    Ballo-Touré        1997-01-03       Fodé   
 13     Milieu      Morocco         Kechta        2002-02-25    Yassine   
 14     Milieu       Russ

In [22]:
list_position = []
for team in team_players:
    for player in team:
        if player['position'] not in list_position:
            list_position.append(player['position'])
list_position

['Goalkeeper',
 'Defence',
 'Midfield',
 'Right Midfield',
 'Right Winger',
 'Left-Back',
 'Attacking Midfield',
 'Centre-Back',
 'Right-Back',
 'Centre-Forward',
 'Central Midfield',
 'Left Winger',
 'Offence',
 'Defensive Midfield',
 'Left Midfield']

In [23]:
for team in team_players:
    for player in team:
        if player['position'] == 'Goalkeeper':
            player['position'] = 'Gardien'
        elif player['position'] in ['Defence', 'Left-Back', 'Right-Back', 'Centre-Back']:
            player['position'] = 'Defenseur'
        elif player['position'] in ['Right Midfield', 'Attacking Midfield', 'Left Midfield', 'Midfield', 'Defensive Midfield', 'Central Midfield']:
            player['position'] = 'Milieu'
        elif player['position'] in ['Right Winger', 'Centre-Forward', 'Left Winger', 'Offence']:
            player['position'] = 'Attaquant'

team_players

[[{'id': 44055,
   'name': 'Kjetil Haug',
   'position': 'Gardien',
   'dateOfBirth': '1998-06-12',
   'nationality': 'Norway'},
  {'id': 178669,
   'name': 'Guillaume Restes',
   'position': 'Gardien',
   'dateOfBirth': '2005-03-11',
   'nationality': 'France'},
  {'id': 212867,
   'name': 'Justin Lacombe',
   'position': 'Gardien',
   'dateOfBirth': '2003-04-09',
   'nationality': 'France'},
  {'id': 274174,
   'name': 'Mathys Niflore',
   'position': 'Gardien',
   'dateOfBirth': '2007-03-02',
   'nationality': 'France'},
  {'id': 260271,
   'name': 'Jaydee Canvot',
   'position': 'Defenseur',
   'dateOfBirth': '2006-07-29',
   'nationality': 'France'},
  {'id': 264401,
   'name': 'Dayann Methalie',
   'position': 'Defenseur',
   'dateOfBirth': '2006-02-15',
   'nationality': 'France'},
  {'id': 260272,
   'name': 'Mathis Saka',
   'position': 'Milieu',
   'dateOfBirth': '2006-09-20',
   'nationality': 'France'},
  {'id': 250994,
   'name': 'Rafik Messali',
   'position': 'Milieu',
 

In [ ]:
try:
    cnx = connector.connect(host='localhost',
                            database='football_predictor',
                            user=config['MYSQL_USER'],
                            password=config['MYSQL_PASSWORD'],
                            )
    if cnx.is_connected():
        print('Connecté à la base MySQL')
        cursor = cnx.cursor()
        # Chargement des données dans la base données SQL
        team_count = 0
        for team in team_players:
            cursor.execute("SELECT id FROM `Team` WHERE shortname = %s;", [team_shortnames[team_count]])
            team_id = cursor.fetchone()
            add_coach_ligue1 = ("INSERT INTO `Staff` (first_name, last_name, birthdate, nationality, role, team_id) "
                                f"VALUES (\"{team_coaches[team_count]['name'].split(' ', 1)[0]}\", "
                                f"\"{team_coaches[team_count]['name'].split(' ', 1)[1]}\", \"{team_coaches[team_count]['dateOfBirth']}\", "
                                f"\"{team_coaches[team_count]['nationality']}\", \"Entraineur\", {team_id[0]})" )
            team_count += 1
            cursor.execute(add_coach_ligue1)

            for player in team:
                add_player_ligue1 = ("INSERT INTO `Player` (first_name, last_name, birthdate, nationality, position, team_id) "
                                    f"VALUES (\"{player['name'].split(' ', 1)[0]}\", \"{player['name'].split(' ', 1)[1]}\", "
                                    f"\"{player['dateOfBirth']}\", \"{player['nationality']}\", \"{player['position']}\", {team_id[0]})" )
                # print(add_player_ligue1)
                cursor.execute(add_player_ligue1)
        for team in df_list:
            cursor.execute("SELECT id FROM `Team` WHERE name = %s;", [team['Equipe'][0]])
            team_id = cursor.fetchone()
            coach = df_coaches_ligue2[df_coaches_ligue2['index'] == team['Equipe'][0]].reset_index()
            add_coach_ligue2 = ("INSERT INTO `Staff` (first_name, last_name, birthdate, nationality, role, team_id) "
                                f"VALUES (\"{coach['surname'][0]}\", \"{coach['name'][0]}\", "
                                f"\"{coach['birthdate'][0]}\", \"{coach['nationality'][0]}\", "
                                f"\"Entraineur\", {team_id[0]})" )
            cursor.execute(add_coach_ligue2)
            for index, row in team.iterrows():
                add_player_ligue2 = ("INSERT INTO `Player` (first_name, last_name, birthdate, nationality, position, team_id) "
                                    f"VALUES (\"{row.iloc[4]}\", \"{row.iloc[2]}\", \"{row.iloc[3]}\", \"{row.iloc[1]}\", "
                                    f"\"{row.iloc[0]}\", {team_id[0]})")
                print(add_player_ligue2)
                cursor.execute(add_player_ligue2)
        cnx.commit()
except Error as e:
    cnx.close()
    print("Erreur lors de la connexion à la base MySQL :", e)
finally:
    if cnx.is_connected():
        cnx.close()
        print('Connexion à la base MySQL fermée')

Connecté à la base MySQL
Toulouse
Brest
Marseille
Montpellier
Lille
Nice
Olympique Lyon
PSG
Stade Rennais
Nantes
RC Lens
Stade de Reims
Monaco
Strasbourg
   level_0                   index surname    name   birthdate nationality
0        0  Le Havre Athletic Club  Didier  Digard  1986-07-12      France
INSERT INTO `Player` (first_name, last_name, birthdate, nationality, position, team_id) VALUES ("Mathieu", "Gorgelin", "1990-08-05 00:00:00", "France", "Gardien", 15)
INSERT INTO `Player` (first_name, last_name, birthdate, nationality, position, team_id) VALUES ("Arthur", "Desmas", "1994-04-07 00:00:00", "France", "Gardien", 15)
INSERT INTO `Player` (first_name, last_name, birthdate, nationality, position, team_id) VALUES ("Gautier", "Lloris", "1995-07-18 00:00:00", "France", "Defenseur", 15)
INSERT INTO `Player` (first_name, last_name, birthdate, nationality, position, team_id) VALUES ("Étienne", "Youté Kinkoué", "2002-01-14 00:00:00", "France", "Defenseur", 15)
INSERT INTO `Player` (fi

In [2]:
df_rankings_list = {}
with open('../data/competition/ligue2.json', 'r') as file:
    rankings_ligue2 = json.load(file)
    for season in rankings_ligue2:
        for key in rankings_ligue2[season]:
            df = pd.DataFrame(rankings_ligue2[season][key], columns=['position', 'link', 'fullname', 'shortname', 'points', 'played', 'win',
                                                                     'draw', 'loss', 'goalFor', 'goalAgainst', 'goalDiff'])
            df_rankings_list[(int(season), key)] = df.drop(columns=['link', 'goalDiff'])
        # df_rankings_ligue2 = pd.read_json('../data/competition/ligue2.json', orient='index')
        # df_rankings_list.append(df_rankings_ligue2)
df_rankings_list

{(2022,
  'TOTAL'):    position                                 fullname               shortname  \
 0         1                   Le Havre Athletic Club             Le Havre AC   
 1         2                    Football Club de Metz                 FC Metz   
 2         3  Football Club des Girondins de Bordeaux   Girondins de Bordeaux   
 3         4                  Sporting Club de Bastia               SC Bastia   
 4         5                      Stade Malherbe Caen                 SM Caen   
 5         6                     En avant de Guingamp             EA Guingamp   
 6         7                      Paris Football Club                Paris FC   
 7         8    Association sportive de Saint-Étienne        AS Saint-Étienne   
 8         9        Football Club Sochaux-Montbéliard  FC Sochaux-Montbéliard   
 9        10                         Grenoble Foot 38        Grenoble Foot 38   
 10       11  Union sportive Quevilly Rouen Métropole       US Quevilly-Rouen   
 11      

In [3]:
df_rankings_list_ligue1 = {}
for season in [2022,2023,2024]:
    with open(f'../data/competition/ligue1_{season}.json', 'r') as file:
        rankings_ligue1 = json.load(file)
        standings_ligue1 = rankings_ligue1['standings']
        df_rankings_list_ligue1[(season, 'HOME')] = pd.DataFrame(standings_ligue1[1]['table']).drop(columns=['form', 'goalDifference'])
        df_rankings_list_ligue1[(season, 'AWAY')] = pd.DataFrame(standings_ligue1[2]['table']).drop(columns=['form', 'goalDifference'])
        df_rankings_list_ligue1[(season, 'TOTAL')] = pd.DataFrame(standings_ligue1[0]['table']).drop(columns=['form', 'goalDifference'])

In [4]:
df_rankings_list_ligue1[(2023,'HOME')]

,position,team,playedGames,won,draw,lost,points,goalsFor,goalsAgainst
0,1,"{'id': 521, 'name': 'Lille OSC', 'shortName': ...",17,11,4,2,37,34,14
1,2,"{'id': 516, 'name': 'Olympique de Marseille', ...",17,10,6,1,36,31,13
2,3,"{'id': 524, 'name': 'Paris Saint-Germain FC', ...",17,9,6,2,33,42,22
3,4,"{'id': 548, 'name': 'AS Monaco FC', 'shortName...",17,10,3,4,33,30,14
4,5,"{'id': 512, 'name': 'Stade Brestois 29', 'shor...",17,8,7,2,31,25,14
5,6,"{'id': 522, 'name': 'OGC Nice', 'shortName': '...",17,9,4,4,31,21,11
6,7,"{'id': 546, 'name': 'Racing Club de Lens', 'sh...",17,9,4,4,31,27,18
7,8,"{'id': 547, 'name': 'Stade de Reims', 'shortNa...",17,8,3,6,27,19,18
8,9,"{'id': 529, 'name': 'Stade Rennais FC 1901', '...",17,7,4,6,25,34,26
9,10,"{'id': 523, 'name': 'Olympique Lyonnais', 'sho...",17,7,4,6,25,25,29


In [5]:
for key, df in df_rankings_list_ligue1.items():
    df['team'] = df['team'].apply(lambda x : dict(x)['shortName'])

In [6]:
df_rankings_list_ligue1[(2022,'HOME')]

,position,team,playedGames,won,draw,lost,points,goalsFor,goalsAgainst
0,1,RC Lens,19,17,1,1,52,41,13
1,2,Stade Rennais,19,15,0,4,45,43,14
2,3,Lille,19,13,4,2,43,40,25
3,4,PSG,19,13,2,4,41,45,25
4,5,Olympique Lyon,19,10,5,4,35,35,19
5,6,Marseille,19,10,4,5,34,35,24
6,7,Lorient,19,9,4,6,31,26,21
7,8,Stade de Reims,19,8,6,5,30,28,23
8,9,Monaco,19,9,3,7,30,37,33
9,10,Clermont Foot,19,9,3,7,30,20,28


In [17]:
try:
    cnx = connector.connect(host='localhost',
                            database='football_predictor',
                            user=config['MYSQL_USER'],
                            password=config['MYSQL_PASSWORD'],
                            )
    if cnx.is_connected():
        print('Connecté à la base MySQL')
        cursor = cnx.cursor()
        # Chargement des données dans la base données SQL
        for season in range(2022, 2025):
            for ligue in ['Ligue 1', 'Ligue 2']:
                cursor.execute("INSERT INTO `League` (name, country, season) VALUES (%s,%s,%s)",
                               [ligue, 'France', season])
                
        for key, df in df_rankings_list_ligue1.items():
            cursor.execute("SELECT id FROM `League` WHERE season = %s AND name = %s;", [key[0], 'Ligue 1'])
            league_id = cursor.fetchone()
            for index, row in df.iterrows():
                cursor.execute("SELECT id FROM `Team` WHERE shortname = %s;", [row['team']])
                team_id = cursor.fetchone()
                if team_id is None:
                    search_similar = f"SELECT id FROM `Team` WHERE shortname LIKE '%{row['team']}%';"
                    cursor.execute(search_similar)
                    team_id = cursor.fetchone()
                add_ranking = ("INSERT INTO `Ranking` (team_id, league_id, type, position, points, played, goals_for, "
                               f"goals_against, won, draw, lost) VALUES ({team_id[0]}, {league_id[0]}, \"{key[1]}\", "
                               f"{row['position']}, {row['points']}, {row['playedGames']}, {row['goalsFor']}, "
                               f"{row['goalsAgainst']}, {row['won']}, {row['draw']}, {row['lost']})")
                cursor.execute(add_ranking)
        for key, df in df_rankings_list.items():
            cursor.execute("SELECT id FROM `League` WHERE season = %s AND name = %s;", [key[0], 'Ligue 2'])
            league_id = cursor.fetchone()
            for index, row in df.iterrows():
                cursor.execute("SELECT id FROM `Team` WHERE shortname = %s;", [row['shortname']])
                team_id = cursor.fetchone()
                if team_id is None:
                    if row['shortname'] == 'US Quevilly Rouen':
                        cursor.execute("SELECT id FROM `Team` WHERE shortname = %s;", ['US Quevilly-Rouen'])
                        team_id = cursor.fetchone()
                add_ranking = ("INSERT INTO `Ranking` (team_id, league_id, type, position, points, played, goals_for, "
                               f"goals_against, won, draw, lost) VALUES ({team_id[0]}, {league_id[0]}, \"{key[1]}\", "
                               f"{row['position']}, {row['points']}, {row['played']}, {row['goalFor']}, "
                               f"{row['goalAgainst']}, {row['win']}, {row['draw']}, {row['loss']})")
        cnx.commit()
except Error as e:
    cnx.close()
    print("Erreur lors de la connexion à la base MySQL :", e)
finally:
    if cnx.is_connected():
        cnx.close()
        print('Connexion à la base MySQL fermée')

Connecté à la base MySQL
Connexion à la base MySQL fermée


In [107]:
client = pymongo.MongoClient('localhost', 27017)
db = client["football_predictor"]
matches_collection = db["matches"]

In [108]:
matches_ligue2 = []
for i in range(2022, 2025):
    with open(f'../data/matches/matches_ligue2_{i}-{i+1}.json') as json_file:
        data = json.load(json_file)
        matches_ligue2.append(data)

In [109]:
DATE_FORMAT = "%A %d %B %Y"
for season in matches_ligue2:
    for matchday in season['season_matches']:
        for matches in matchday['matches']:
            matches['date'] = dt.datetime.strptime(matches['date'], DATE_FORMAT).strftime('%Y-%m-%d')

In [110]:
matches_mongo = []
for season in matches_ligue2:
    for matchday in season['season_matches']:
        for matches in matchday['matches']:
            matches_mongo.append({'league' : 'Ligue 2',
                                  'season' : season['season'],
                                  'matchday' : matchday['matchday'],
                                  'matches' : matches
                                 })

In [111]:
matches_collection.insert_many(matches_mongo)

InsertManyResult([ObjectId('67c5b27a49b164caafca4e24'), ObjectId('67c5b27a49b164caafca4e25'), ObjectId('67c5b27a49b164caafca4e26'), ObjectId('67c5b27a49b164caafca4e27'), ObjectId('67c5b27a49b164caafca4e28'), ObjectId('67c5b27a49b164caafca4e29'), ObjectId('67c5b27a49b164caafca4e2a'), ObjectId('67c5b27a49b164caafca4e2b'), ObjectId('67c5b27a49b164caafca4e2c'), ObjectId('67c5b27a49b164caafca4e2d'), ObjectId('67c5b27a49b164caafca4e2e'), ObjectId('67c5b27a49b164caafca4e2f'), ObjectId('67c5b27a49b164caafca4e30'), ObjectId('67c5b27a49b164caafca4e31'), ObjectId('67c5b27a49b164caafca4e32'), ObjectId('67c5b27a49b164caafca4e33'), ObjectId('67c5b27a49b164caafca4e34'), ObjectId('67c5b27a49b164caafca4e35'), ObjectId('67c5b27a49b164caafca4e36'), ObjectId('67c5b27a49b164caafca4e37'), ObjectId('67c5b27a49b164caafca4e38'), ObjectId('67c5b27a49b164caafca4e39'), ObjectId('67c5b27a49b164caafca4e3a'), ObjectId('67c5b27a49b164caafca4e3b'), ObjectId('67c5b27a49b164caafca4e3c'), ObjectId('67c5b27a49b164caafca4e

In [112]:
matches_ligue1 = []
for i in range(2022, 2025):
    matchnumber = 34
    if i == 2022:
        matchnumber = 38
    for j in range(matchnumber):
        with open(f'../data/matches/season-{i}_matches-{j+1}.json') as json_file:
            data = json.load(json_file)
            matches_ligue1.append(data)

In [113]:
matches_mongo = []
for matchday in matches_ligue1:
    matches_temp = []
    for match in matchday['matches']:
        matches_temp.append({ 'date' : match['utcDate'][:10],
                              'home_team' : match['homeTeam']['shortName'],
                              'away_team' : match['awayTeam']['shortName'],
                              'score_halftime' : match['score']['halfTime'],
                              'score' : match['score']['fullTime']})
    matches_mongo.append({ 'league' : matchday['competition']['name'],
                           'season' : f"{matchday['filters']['season']}-{int(matchday['filters']['season']+1)}",
                           'matchday' : int(matchday['filters']['matchday']),
                           'matches' : matches_temp})

In [114]:
matches_collection.insert_many(matches_mongo)

InsertManyResult([ObjectId('67c5b27d49b164caafca4ef9'), ObjectId('67c5b27d49b164caafca4efa'), ObjectId('67c5b27d49b164caafca4efb'), ObjectId('67c5b27d49b164caafca4efc'), ObjectId('67c5b27d49b164caafca4efd'), ObjectId('67c5b27d49b164caafca4efe'), ObjectId('67c5b27d49b164caafca4eff'), ObjectId('67c5b27d49b164caafca4f00'), ObjectId('67c5b27d49b164caafca4f01'), ObjectId('67c5b27d49b164caafca4f02'), ObjectId('67c5b27d49b164caafca4f03'), ObjectId('67c5b27d49b164caafca4f04'), ObjectId('67c5b27d49b164caafca4f05'), ObjectId('67c5b27d49b164caafca4f06'), ObjectId('67c5b27d49b164caafca4f07'), ObjectId('67c5b27d49b164caafca4f08'), ObjectId('67c5b27d49b164caafca4f09'), ObjectId('67c5b27d49b164caafca4f0a'), ObjectId('67c5b27d49b164caafca4f0b'), ObjectId('67c5b27d49b164caafca4f0c'), ObjectId('67c5b27d49b164caafca4f0d'), ObjectId('67c5b27d49b164caafca4f0e'), ObjectId('67c5b27d49b164caafca4f0f'), ObjectId('67c5b27d49b164caafca4f10'), ObjectId('67c5b27d49b164caafca4f11'), ObjectId('67c5b27d49b164caafca4f

In [115]:
matchday_cursor = matches_collection.find({"league" : "Ligue 1"})

In [116]:
try:
    cnx = connector.connect(host='localhost',
                            database='football_predictor',
                            user=config['MYSQL_USER'],
                            password=config['MYSQL_PASSWORD'],
                            )
    if cnx.is_connected():
        print('Connecté à la base MySQL')
        cursor = cnx.cursor()
        # Chargement des données dans la base données SQL
        for matchday in matchday_cursor:
            i = 0
            for match in matchday['matches']:
                search_similar_home = f"SELECT id FROM Team WHERE shortname LIKE '%{match['home_team']}%'"
                cursor.execute(search_similar_home)
                home_team_id = cursor.fetchone()
                search_similar_away = f"SELECT id FROM Team WHERE shortname LIKE '%{match['away_team']}%'"
                cursor.execute(search_similar_away)
                away_team_id = cursor.fetchone()
                team_update = { '$set' : {
                                            f'matches.{i}' : {'date' : match['date'],
                                                              'home_team' : {'id' : int(home_team_id[0]), 'name' : match['home_team']},
                                                              'away_team' : {'id' : int(away_team_id[0]), 'name' : match['away_team']},
                                                              'score_halftime' : match['score_halftime'],
                                                              'score' : match['score']}
                                         }
                              }
                i += 1
                matches_collection.update_one({'_id' : matchday['_id']}, team_update)
    cnx.commit()
except Error as e:
    cnx.close()
    print("Erreur lors de la connexion à la base MySQL :", e)
finally:
    if cnx.is_connected():
        cnx.close()
        print('Connexion à la base MySQL fermée')

Connecté à la base MySQL
Connexion à la base MySQL fermée


In [117]:
matchday_cursor = matches_collection.find({"league" : "Ligue 2"})

In [118]:
try:
    cnx = connector.connect(host='localhost',
                            database='football_predictor',
                            user=config['MYSQL_USER'],
                            password=config['MYSQL_PASSWORD'],
                            )
    if cnx.is_connected():
        print('Connecté à la base MySQL')
        cursor = cnx.cursor()
        # Chargement des données dans la base données SQL
        for matchday in matchday_cursor:
            i = 0
            for match in matchday['matches']['results']:
                search_similar_home = f"SELECT id FROM Team WHERE shortname LIKE '%{match['home_team']}%'"
                cursor.execute(search_similar_home)
                home_team_id = cursor.fetchone()
                search_similar_away = f"SELECT id FROM Team WHERE shortname LIKE '%{match['away_team']}%'"
                cursor.execute(search_similar_away)
                away_team_id = cursor.fetchone()
                team_update = { '$set' : {
                                            f'matches.results.{i}' : {
                                                                'home_team' : {'id' : int(home_team_id[0]), 'name' : match['home_team']},
                                                                'score' : match['score'],
                                                                'away_team' : {'id' : int(away_team_id[0]), 'name' : match['away_team']}
                                                        }
                                        }
                            }
                i += 1
                matches_collection.update_one({'_id' : matchday['_id']}, team_update)
    cnx.commit()
except Error as e:
    cnx.close()
    print("Erreur lors de la connexion à la base MySQL :", e)
finally:
    if cnx.is_connected():
        cnx.close()
        print('Connexion à la base MySQL fermée')

Connecté à la base MySQL
Connexion à la base MySQL fermée
